In [1]:
from caffe import layers as L, params as P
import caffe

In [2]:
caffe.set_mode_gpu()
caffe.set_device(1)

In [5]:
def net(data_set, batch_size, scope):
    n = caffe.NetSpec()
    n.data, n.label = L.ImageData(
        batch_size=batch_size, 
        source=data_set, 
        is_color=False, 
        transform_param=dict(crop_size=100), 
        ntop=2)
    
    n.conv1_1 = L.Convolution(n.data, kernel_size=3, pad=1, num_output=32, weight_filler=dict(type='xavier'))
    n.batch1_1 = L.BatchNorm(n.conv1_1)
    n.relu1_1 = L.ReLU(n.batch1_1, in_place=True)
    n.conv1_2 = L.Convolution(n.relu1_1, kernel_size=3, pad=1, num_output=32, weight_filler=dict(type='xavier'))
    n.batch1_2 = L.BatchNorm(n.conv1_2)
    n.relu1_2 = L.ReLU(n.batch1_2, in_place=True)
    n.pool1 = L.Pooling(n.relu1_2, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    
    n.conv2_1 = L.Convolution(n.pool1, kernel_size=3, pad=1, num_output=64, weight_filler=dict(type='xavier'))
    n.batch2_1 = L.BatchNorm(n.conv2_1)
    n.relu2_1 = L.ReLU(n.batch2_1, in_place=True)
    n.conv2_2 = L.Convolution(n.relu2_1, kernel_size=3, pad=1, num_output=64, weight_filler=dict(type='xavier'))
    n.batch2_2 = L.BatchNorm(n.conv2_2)
    n.relu2_2 = L.ReLU(n.batch2_2, in_place=True)
    n.pool2 = L.Pooling(n.relu2_2, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    
    n.conv3_1 = L.Convolution(n.pool2, kernel_size=3, pad=1, num_output=128, weight_filler=dict(type='xavier'))
    n.batch3_1 = L.BatchNorm(n.conv3_1)
    n.relu3_1 = L.ReLU(n.batch3_1, in_place=True)
    n.conv3_2 = L.Convolution(n.relu3_1, kernel_size=3, pad=1, num_output=128, weight_filler=dict(type='xavier'))
    n.batch3_2 = L.BatchNorm(n.conv3_2)
    n.relu3_2 = L.ReLU(n.batch3_2, in_place=True)
    n.pool3 = L.Pooling(n.relu3_2, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    
    n.conv4_1 = L.Convolution(n.pool3, kernel_size=3, pad=1, num_output=256, weight_filler=dict(type='xavier'))
    n.batch4_1 = L.BatchNorm(n.conv4_1)
    n.relu4_1 = L.ReLU(n.batch4_1, in_place=True)
    n.pool4 = L.Pooling(n.relu4_1, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    
    n.fc1 =   L.InnerProduct(n.pool4, num_output=1000, weight_filler=dict(type='xavier'))
    n.relu_a1 = L.ReLU(n.fc1, in_place=True)
    n.fc2 = L.InnerProduct(n.relu_a1, num_output=500, weight_filler=dict(type='xavier'))
    n.relu_a2 = L.ReLU(n.fc2, in_place=True)
    n.fc3 = L.InnerProduct(n.relu_a2, num_output=200, weight_filler=dict(type='xavier'))
    n.relu_a3 = L.ReLU(n.fc3, in_place=True)
    
    n.score = L.InnerProduct(n.relu_a3, num_output=11, weight_filler=dict(type='xavier'))
    n.loss =  L.SoftmaxWithLoss(n.score, n.label)
    
    n.accuracy_top1 = L.Accuracy(n.score, n.label)
    #n.accuracy_top5 = L.Accuracy(n.score, n.label, accuracy_param=dict(top_k=5))
        
    return n.to_proto()
    
with open('proto/kilbas_train.prototxt', 'w') as f:
    f.write(str(net('resources/train.txt', 200, 'TRAIN')))

with open('proto/kilbas_test.prototxt', 'w') as f:
    f.write(str(net('resources/test.txt', 200, 'TEST')))

In [6]:
solver = caffe.SGDSolver('proto/kilbas_solver.prototxt')
[(k, v.data.shape) for k, v in solver.net.blobs.items()]

[('data', (200, 1, 100, 100)),
 ('label', (200,)),
 ('label_data_1_split_0', (200,)),
 ('label_data_1_split_1', (200,)),
 ('conv1_1', (200, 32, 100, 100)),
 ('batch1_1', (200, 32, 100, 100)),
 ('conv1_2', (200, 32, 100, 100)),
 ('batch1_2', (200, 32, 100, 100)),
 ('pool1', (200, 32, 50, 50)),
 ('conv2_1', (200, 64, 50, 50)),
 ('batch2_1', (200, 64, 50, 50)),
 ('conv2_2', (200, 64, 50, 50)),
 ('batch2_2', (200, 64, 50, 50)),
 ('pool2', (200, 64, 25, 25)),
 ('conv3_1', (200, 128, 25, 25)),
 ('batch3_1', (200, 128, 25, 25)),
 ('conv3_2', (200, 128, 25, 25)),
 ('batch3_2', (200, 128, 25, 25)),
 ('pool3', (200, 128, 13, 13)),
 ('conv4_1', (200, 256, 13, 13)),
 ('batch4_1', (200, 256, 13, 13)),
 ('pool4', (200, 256, 7, 7)),
 ('fc1', (200, 1000)),
 ('fc2', (200, 500)),
 ('fc3', (200, 200)),
 ('score', (200, 11)),
 ('score_score_0_split_0', (200, 11)),
 ('score_score_0_split_1', (200, 11)),
 ('loss', ()),
 ('accuracy_top1', ())]

In [ ]:
solver.solve()

23000 эпох SGD with moment

base_lr:0.00001
lr_policy:"inv"
power:0.75
gamma:0.0001

weight_decay:0.0001
momentum: 0.9

точность ~71%